In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.simplefilter('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util
from preprocessor import Processor

In [3]:
util = Util()
cleaner = DataCleaner()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Natnael\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
news_df=util.read_from_dvc("data/news.csv",repo,"news-v0",low_memory=False)

# cleaned_df=util.read_from_file('../data/news.csv')

In [5]:
news_df=news_df.sample(frac=1)

train_news=news_df.head(8)
test_news=news_df.tail(2)

In [6]:
full_processed_df=processor.prepare_text(train_news.copy())
description_processed=processor.prepare_text(train_news.copy(),columns=["Description"])[["Description","Analyst_Average_Score"]]
title_processed=processor.prepare_text(train_news.copy(),columns=["Title"])[["Title","Analyst_Average_Score"]]
body_processed=processor.prepare_text(train_news.copy(),columns=["Body"])[["Body","Analyst_Average_Score"]]
unprocessed=train_news

processed_test=processor.prepare_text(test_news)

In [8]:
full_processed_df['Description']=full_processed_df['Title'] + ", " + full_processed_df['Description'] + ", " + full_processed_df["Body"]
full_processed_df.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)

title_processed.rename(columns={'Title':'Description'},inplace=True)

body_processed.rename(columns={'Body':'Description'},inplace=True)

unprocessed['Description']=unprocessed['Title'] + ", " + unprocessed['Description'] + ", " + unprocessed["Body"]
unprocessed.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)

processed_test['Description']=processed_test['Title'] + ", " + processed_test['Description'] + ", " + processed_test["Body"]
processed_test.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)




In [11]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)


In [15]:
full_promp_trainer=full_promp_trainer.reset_index().drop(['index'],axis=1)

In [16]:
# full_promp_trainer=full_promp_trainer[['Domain','Title','Description','Body','timestamp','Analyst_Average_Score']]
full_promp_trainer.rename(columns={"Analyst_Average_Score":"Analyst Average Score"},inplace=True)
# full_promp_trainer=cleaner.convert_to_datetime(full_promp_trainer,['timestamp'])
# full_promp_trainer=full_promp_trainer.fillna('')
full_promp_trainer.head()

,Description,Analyst Average Score
0,"jet, thing, take, fund, johnson, taxpay, histo...",0.00
1,"collaps, scoan, nigeria, 7th, anniversari, sab...",0.00
2,"male, murder, arrest, femal, sap, 2021, 22, 13...",1.33
3,"woman, eskom, tweet, news24, dismiss, punish, ...",0.33
4,"construct, expect, repair, sector, loot, boost...",1.66


In [17]:

processor.prepare_tuner(full_promp_trainer)

tuner prepared successfuly


In [18]:
# uncomment these for first time use
# full_processed_df.to_csv("../data/trainer_news.csv",index=False)
# processed_test.to_csv("../data/test_news.csv",index=False)